<a href="https://colab.research.google.com/github/gladcolor/GEOG105_Fall2022/blob/master/GEOG105_Geemap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installl the package: `geemap` 

Need to upgrade and restart the runtime (i.e., the running environment). 

`!` means run the system command line in the current cell of Jupyter Notebook.

In [3]:
! pip install git+https://github.com/giswqs/geemap
! upgrade geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/giswqs/geemap to /tmp/pip-req-build-xnzfxysu
  Running command git clone -q https://github.com/giswqs/geemap /tmp/pip-req-build-xnzfxysu
     |████████████████████████████████| 1.2 MB 8.8 MB/s 
     |████████████████████████████████| 198 kB 58.9 MB/s 
     |████████████████████████████████| 98 kB 7.3 MB/s 
     |████████████████████████████████| 130 kB 59.6 MB/s 
     |████████████████████████████████| 3.4 MB 23.8 MB/s 
     |████████████████████████████████| 1.3 MB 69.7 MB/s 
     |████████████████████████████████| 8.8 MB 28.6 MB/s 
     |████████████████████████████████| 46 kB 3.2 MB/s 
     |████████████████████████████████| 3.0 MB 24.9 MB/s 
     |████████████████████████████████| 99 kB 7.7 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 128 kB 71.1 MB/s 
     |████████████████████████████████| 1.6 MB 3

/bin/bash: upgrade: command not found


In [156]:
import geemap
import ee

Map = geemap.Map(height=700)  # creat a new map view, and its height is 700 pixel.

# show the map, make sure the environment is set up correctly
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Copy the Google Earth Engine Token here for further use: (e.g., 4/1ARtbsJpPd6r5a-g026KUyLkKkzW1DoahkWWaCVFkDJYe10oKalQKecJt2HY)

# Make a test to see whether NAIP can be loaded into the map.

In [157]:
dataset = ee.ImageCollection('USDA/NAIP/DOQQ') \
                  .filter(ee.Filter.date('2017-01-01', '2018-12-31'))
TrueColor = dataset.select(['R', 'G', 'B'])  
TrueColorVis = {
  'min': 0.0,
  'max': 255.0,
}
Map.setCenter(-73.9958, 40.7278, 15)
Map.addLayer(TrueColor, TrueColorVis, 'NAIP True Color')

Map

Map(bottom=455.0, center=[40.7278, -73.9958], controls=(WidgetControl(options=['position', 'transparent_bg'], …

# Compute Normalized Difference Vegetation Index (NDVI)  using NAIP imagery. 

NDVI is a common index in remote sensing imagery analysis. NDVI always ranges from -1 to +1. A high NDVI value close to +1 indicates a high possibility that it’s dense green leaves. A low NDVI value close to 0 or even -1 means that the pixel is probably not vegetation, such as water or impervious surface.



$ NDVI = \frac{NIR-Red}{NIR+Red}$

`NIR`: pixel value of Near-infrared band

`Red`: pixel value of Red band



##  Obtain the NAIP image

The latest NAIP image has a high resolution of 0.6 m; the old ones are 1 m.

In [159]:
import ee

# the bounding box of Columbia City:
west =-81.1340398311
south = 33.9615728841
east = -80.97881095
north = 34.0333482753

dataset = ee.ImageCollection('USDA/NAIP/DOQQ') \
                  .filterBounds(ee.Geometry.Rectangle(west, south, east, north)) \
                  .filter(ee.Filter.date('2019-01-01', '2022-08-31') \
                  )
                  # USDA/NAIP/DOQQ: name of the image collection; NAIP: National Agriculture Imagery Program
                  # Rectangle:  west, south, east, north
                  # ee.Filter.date: filter by date

In [160]:
nir = dataset.select("N").mosaic()  # extract the near-infrared band
red = dataset.select("R").mosaic()  # extract the red band

# compute the NDVI
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
# ndvi is image.

# set the parameters to show the NDVI image. Recall the range of NDVI is [-1, 1]
ndviParams = {"min": -1, "max": 1, "palette": ['blue', 'white', 'green']} 

Map.addLayer(ee_object=ndvi, vis_params=ndviParams, shown=True, opacity=1, name='NDVI')

Map.setCenter(lon=-80.9977, lat=33.9579, zoom=11)

Map

Map(bottom=424670.0, center=[33.9579, -80.9977], controls=(WidgetControl(options=['position', 'transparent_bg'…

## Export the NDVI image

In [161]:
roi = ee.Geometry.BBox(west, south, east, north)  # create a Bounding box

geemap.ee_export_image_to_drive(image=ndvi, description='Columbia_ndvi_10m', folder='NDVI_export', scale=10, region=roi)

# scale: Resolution in meters per pixel. Defaults to 1000.
# description: (almost is file name) A human-readable name of the task. May contain letters, numbers, -, _ (no spaces). 

# wait about 2 minutes, then you can find Columbia_ndvi_10m.tif is in the NDVI_export folder. 
# If you need to export large area of data, please use JavaScrit API (https://code.earthengine.google.com/).

In [ ]:
## Export the True color NAIP image

In [164]:
# Export NAIP true color image
geemap.ee_export_image_to_drive(image=TrueColor.mosaic(), description='Columbia_true_color_5m', folder='NDVI_export', scale=5, region=roi)

# wait about 3 minutes, then you can find Columbia_ndvi_10m.tif is in the NDVI_export folder. 
